In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import sympy

In [2]:
wavelength = 1
Ns = 3
dy = 2 * wavelength
Lz = 5 * wavelength
Nr = 3
k = 2 * np.pi / wavelength

In [6]:
def euclidean_distance(s, d) -> float:
    # s -> source point
    # d -> destination (receiving point)
    s = np.asarray(s)
    d = np.asarray(d)
    return np.linalg.norm(s - d)

def free_space_transfer_function(r_s, r_r, k) -> np.array:
    # r_s -> matrix of source points, shape (Ns, 2)
    # r_r -> matrix of recieving points, shape (Nr, 2)
    # k -> wavenumber
    # returns g -> matrix of floats, shape (Nr, Ns)
    g = np.zeros((len(r_r), len(r_s)), dtype=complex)
    for i, r in enumerate(r_r):
        for j, s in enumerate(r_s):
            d = euclidean_distance(s, r)
            # Check for eq 39
            if j == 2 and i == 0:
                check_d = d
            numerator = -np.exp(1j * k * d)
            denominator = 4 * np.pi * d
            g[i,j] = np.round(numerator / denominator, 5) # rounding here is questionable, but I need it for the checks
    return g, check_d

def sum_rule(g: np.ndarray) -> float:
    return np.linalg.norm(g, 'fro')**2

def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")

In [7]:
s = np.asarray([(0, i * 2 * wavelength) for i in range(Ns)])
d = np.asarray([(5 * wavelength, i * 2 * wavelength) for i in range(Nr)])
Gsr, check_d = free_space_transfer_function(s, d, k)

# eq 39 check
print(np.allclose(np.sqrt(41) * wavelength, check_d))

# eq 40 check
print(np.allclose((0.01020 - 1j*0.00711)/wavelength, Gsr[0,2]))

# Convert to units of wavelength
gsr = np.round(-4 * np.pi * Lz, 2)
gsr_Gsr = np.round(gsr*Gsr, 2)

# Compares to eq 43
print(gsr_Gsr)

True
True
[[ 1.  -0.j   -0.7 +0.61j -0.64+0.45j]
 [-0.7 +0.61j  1.  -0.j   -0.7 +0.61j]
 [-0.64+0.45j -0.7 +0.61j  1.  -0.j  ]]


In [8]:
print(Gsr.shape)

(3, 3)


In [9]:
S = sum_rule(Gsr)

# eq 44 check
print(np.allclose(S, 7.67 / gsr**2, atol=1e-6))

True


In [6]:
gsr_Gsrd_Gsr = np.round(gsr**2 * np.matmul(np.matrix.getH(Gsr), Gsr), 2)

temp = np.array(
    [[2.47, -0.67 - 1j*0.08, -0.42],
    [-0.67 + 1j*0.08, 2.72, -0.67 + 1j*0.08],
    [-0.42, -0.67 - 1j*0.08, 2.47]], dtype=complex)

# eq 45 check
print(np.allclose(gsr_Gsrd_Gsr, temp))


True


In [7]:
# Check that gsr_Gsrd_Gsr is Hermetian
temp = np.matrix.getH(gsr_Gsrd_Gsr)
print((temp == gsr_Gsrd_Gsr).all())

True


In [16]:
# Now for eigenvalues / eigenvectors
Gsrd_Gsr = np.matmul(np.matrix.getH(Gsr), Gsr)
Gsr_Gsrd = np.matmul(Gsr, np.matrix.getH(Gsr))
from scipy.linalg import eigh
eig_vals_0, eig_vect_0 = eigh(Gsrd_Gsr)
eig_vals_1, eig_vect_1 = np.linalg.eigh(Gsrd_Gsr)
eig_vals_2, eig_vect_2 = np.linalg.eigh(Gsr_Gsrd)
print(np.allclose(eig_vals_0, eig_vals_1))

# Both scipy and np return the eigen values in opposite order
eig_vals_0 = np.round(np.flip(eig_vals_0), 5)
eig_vect_0 = np.round(np.flip(eig_vect_0), 2)

eig_vals_1 = np.round(np.flip(eig_vals_1), 5)
eig_vect_1 = np.round(np.flip(eig_vect_1), 2)

eig_vals_2 = np.round(np.flip(eig_vals_2), 5)
eig_vect_2 = np.round(np.flip(eig_vect_2), 2)

True


In [17]:
matprint(eig_vect_1)
print()
matprint(eig_vect_2)

   0.41+0j  -0.71-0j     -0.58+0j  
-0.81+0.1j      0+0j  -0.57+0.07j  
   0.41+0j   0.71+0j     -0.58+0j  

   0.41-0j  -0.71+0j     -0.58-0j  
-0.81-0.1j      0+0j  -0.57-0.07j  
   0.41+0j   0.71+0j     -0.58+0j  


In [10]:
# eq 46 check
temp = np.round(np.asarray([3.41, 2.89, 1.37]) / gsr**2, 5)
print(np.allclose(eig_vals_0, temp))
print(np.allclose(eig_vals_2, temp))

True
True


In [11]:
# eq 47 check
temp = np.round(np.sum(eig_vals_0), 5)
print(np.allclose(np.round(S, 5), np.round(temp, 5)))

temp = np.round(np.sum(eig_vals_2), 5)
print(np.allclose(np.round(S, 5), np.round(temp, 5)))

True
True


In [12]:
# eq 48 check
print(eig_vect_0)
print()
print(eig_vect_2)

# Here, the first eigenvectors are out of phase with what is shown in the paper.
# This is mentioned in the note 138 of the references. However, the second ones
# are the same phase. IDK.

[[-0.41-0.j   -0.71-0.j   -0.58+0.j  ]
 [ 0.81-0.1j  -0.  +0.j   -0.57+0.07j]
 [-0.41-0.j    0.71+0.j   -0.58-0.j  ]]

[[ 0.41-0.j   -0.71+0.j   -0.58-0.j  ]
 [-0.81-0.1j   0.  +0.j   -0.57-0.07j]
 [ 0.41+0.j    0.71+0.j   -0.58+0.j  ]]
